# Step1: Connect to AWS DynamoDB database - "test_candidates" table

- AWS has limited size of return responses so we won't get all candidates in one time table scanning
- if 'LastEvaluatedKey' presents in response, keep scaning table to get all candidates 

In [1]:
import boto3
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('test_candidates')
response = table.scan()
data = response['Items']
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [2]:
import pandas as pd
data = pd.DataFrame(data)
# data = data[:len(data)//2]

# Step2: Data exploration

In [4]:
# pandas dataframe
# data.to_csv("test_candidate.csv")  # in case you want to export to csv
data

address  \
0                                                   NaN   
1                                                   nan   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
5                  111/48 Moo7 Suanyai Muang NonThaburi   
6                                                   NaN   
7                                                   NaN   
8                                                   nan   
9                                                   nan   
10                                                  NaN   
11                                                  nan   
12                                                  NaN   
13                                                  nan   
14                                                  nan   
15                                                  nan   
16                                                  nan   
17                                                  nan   
18                                                  nan   
19                                                  nan   
20                                                  NaN   
21                                                  nan   
22                                                  NaN   
23                                                  nan   
24                                                  nan   
25                                                  nan   
26                                                  nan   
27                                                  nan   
28                                                  nan   
29                                                  NaN   
...                                                 ...   
2835                                                nan   
2836                                                nan   
2837                                                NaN   
2838                                                nan   
2839                                                nan   
2840                                                nan   
2841                                                nan   
2842                                                nan   
2843                                                nan   
2844  888/39, Sittharom village, Sukhumvit 105, Bang...   
2845                                                nan   
2846                                                nan   
2847                                                nan   
2848                                                nan   
2849                                                nan   
2850                                                NaN   
2851                                                NaN   
2852                                                NaN   
2853                                                NaN   
2854                                                nan   
2855                                                nan   
2856                                jeedy_b@hotmail.com   
2857  Julius Valentiners Vej 41 DK 2000 Frederiksber...   
2858                                                nan   
2859                                                NaN   
2860                                                nan   
2861                                                NaN   
2862                                                NaN   
2863                                                nan   
2864                                                NaN   

                                             alive_dead  \
0                                                   NaN   
1                                                   N/A   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
5                                  

In [7]:
# all data columns
print(data.columns)
print("there are {} columns in this dataframe".format(len(data.columns)))

Index(['address', 'alive_dead', 'avatar', 'avatar_url', 'birthday', 'bonus',
       'consultant', 'country', 'current_salary', 'cv_link', 'devpost_url',
       'educations', 'email', 'followers', 'followings', 'full_name',
       'github_created_at', 'github_followers', 'github_followings',
       'github_public_gists', 'github_public_repos', 'github_updated_at',
       'github_url', 'hirable', 'jbscore', 'jobs', 'kaggle_url', 'last_update',
       'linkedin_url', 'location', 'messengers', 'neighborhood',
       'organizations', 'ownership', 'phones', 'profile_url', 'public_gists',
       'public_repos', 'reason_for_leaving', 'repos_fork', 'repos_forks_count',
       'repos_languages', 'salary', 'skills', 'stargazers_count', 'status',
       'summary', 'target_company', 'target_position', 'time_spent_each_job',
       'title', 'twitter', 'uuid', 'websites'],
      dtype='object')
there are 54 columns in this dataframe


# Step3: Feature selection, create a clone of original data 

In [11]:
test_data = data.copy()

In [27]:
test_data = data.copy()
# TODO later we should incorporate these features to get insightful/hidden information
# 3.1 remove unnessesary columns 
del test_data["websites"]
del test_data["twitter"]
del test_data["alive_dead"]
del test_data["address"]
del test_data["avatar"]
del test_data["avatar_url"]
del test_data["birthday"]
del test_data["cv_link"]
del test_data["devpost_url"]
del test_data["email"]
del test_data["full_name"]
del test_data["kaggle_url"]
del test_data["github_created_at"]
del test_data["last_update"]
del test_data["profile_url"]
del test_data["phones"]
del test_data["linkedin_url"]
del test_data["messengers"]
del test_data["neighborhood"]
del test_data["current_salary"]
del test_data["time_spent_each_job"]
del test_data["status"]
del test_data["stargazers_count"]
del test_data["repos_languages"]
del test_data["repos_fork"]
del test_data["consultant"]
del test_data["bonus"]
del test_data["followers"]
del test_data["followings"]
del test_data["github_url"]
del test_data["jbscore"]
del test_data["public_repos"]
del test_data["repos_forks_count"]
del test_data["uuid"]
del test_data["github_updated_at"]
del test_data["country"]
del test_data["hirable"]
del test_data["public_gists"]
del test_data["ownership"]
del test_data["target_company"]
del test_data["target_position"]
del test_data["reason_for_leaving"]

# 3.2 check if skills column has null value
item = test_data["skills"].isnull()
drop_list = []
for i in range(len(item)):
    # 3.2.1 candidates with null skills value
    if item[i] == True:
        drop_list.append(i)
        
test_data = test_data.drop(test_data.index[drop_list])
print("there are {} candidates with null \"skill\"".format(len(test_data)))
test_data.head()

there are 1892 candidates with null "skill"


educations github_followers  \
0                                                 []              NaN   
1                                                 []              NaN   
5  [{'Education 1': {'Education Start 1': '1999.0...              NaN   
8  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
9  [{'Education 1': {'Education Start 1': '2012.0...              NaN   

  github_followings github_public_gists github_public_repos jobs  \
0               NaN                 NaN                 NaN  NaN   
1               NaN                 NaN                 NaN  NaN   
5               NaN                 NaN                 NaN  NaN   
8               NaN                 NaN                 NaN  NaN   
9               NaN                 NaN                 NaN  NaN   

                                  location  \
0                                  Bangkok   
1      Bangkok Metropolitan Area, Thailand   
5  Mueang Nonthaburi, Nonthaburi, Thailand   
8      Bangkok Metropolitan Area, Thailand   
9           Bangkok Noi, Bangkok, Thailand   

                                       organizations salary skills  \
0                                                 []    NaN     []   
1                                                 []    NaN     []   
5  [{'Organization 1': {'Organization End 1': 'PR...    N/A     []   
8  [{'Organization 1': {'Organization End 1': 'No...    N/A     []   
9  [{'Organization 1': {'Organization End 1': 'PR...    N/A     []   

                                             summary  \
0                                                NaN   
1                                              29/09   
5                                                nan   
8                                                nan   
9  TOEIC 655, Learning SPSS, Interest in Leadersh...   

                                               title  
0                                                NaN  
1                             Internal Audit Manager  
5  HR Overseas Support Manager  at SCG Packaging ...  
8                             Recruitment Specialist  
9                 Student at Ramkhamhaeng University

In [28]:
# test_data.head()
test_data = test_data.reset_index(drop=True)
test_data

educations github_followers  \
0                                                    []              NaN   
1                                                    []              NaN   
2     [{'Education 1': {'Education Start 1': '1999.0...              NaN   
3     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
4     [{'Education 1': {'Education Start 1': '2012.0...              NaN   
5                                                    []              NaN   
6     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
7     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
8     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
9     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
10    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
11    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
12    [{'Education 1': {'Education Start 1': '2000.0...              NaN   
13    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
14    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
15    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
16    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
17    [{'Education 1': {'Education Start 1': '2008.0...              NaN   
18    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
19    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
20    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
21    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
22    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
23                                                   []              NaN   
24    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
25    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
26    [{'Education 1': {'Education Start 1': '2007.0...              NaN   
27    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
28    [{'Education 1': {'Education Start 1': '2003.0...              NaN   
29    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
...                                                 ...              ...   
1862  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1863  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1864  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1865  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1866  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1867  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1868  [{'Education 1': {'Education Start 1': '2009.0...              NaN   
1869  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1870  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1871  [{'Education 1': {'Education Start 1': '1998.0...              NaN   
1872  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1873  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1874  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1875  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1876  [{'Education 1': {'Education Start 1': '2001.0...              NaN   
1877  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1878  [{'Education 1': {'Education Start 1': '2008.0...              NaN   
1879  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1880  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1881  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1882  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1883  [{'Educatio

In [34]:
# 3.3 drop columns where there is no skills
drop_list = []
for i in range(len(test_data["skills"])):
    if len(test_data.loc[i]["skills"]) == 0:
        drop_list.append(i)
test_data = test_data.drop(test_data.index[drop_list])
test_data = test_data.reset_index(drop=True)
print("there are {} candidates with not null skills".format(len(test_data)))
test_data

there are 1203 candidates with not null skills


educations github_followers  \
0     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
2     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
3     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
4     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
5     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
6     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
7     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
8     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
9     [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
10    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
11    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
12    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
13    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
14    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
15    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
16    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
17    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
18    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
19    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
20    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
21    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
22    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
23    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
24    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
25    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
26    [{'Education 1': {'Education Start 1': '1999.0...              NaN   
27    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
28    [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
29    [{'Education 1': {'Education Start 1': '2008.0...              NaN   
...                                                 ...              ...   
1173  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1174  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1175  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1176  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1177  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1178  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1179  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1180  [{'Education 1': {'Education Start 1': '2008.0...              NaN   
1181  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1182  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1183  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1184  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1185  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1186  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1187  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1188  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1189  [{'Education 1': {'Education Start 1': '2009.0...              NaN   
1190  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1191  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1192  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1193  [{'Education 1': {'Education Start 1': 'nan', ...              NaN   
1194  [{'Educatio

In [35]:
# TODO fill in data for candidates to include github, 
# 3.4 drop columns where values are null
drop_data = test_data.copy()
del drop_data["github_followers"]
del drop_data["github_followings"]
del drop_data["github_public_gists"]
del drop_data["github_public_repos"]
del drop_data["salary"]
del drop_data["summary"]

In [36]:
drop_data

educations jobs  \
0     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
2     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
3     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
4     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
5     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
6     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
7     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
8     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
9     [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
10    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
11    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
12    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
13    [{'Education 1': {'Education Start 1': 'nan', ...   []   
14    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
15    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
16    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
17    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
18    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
19    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
20    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
21    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
22    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
23    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
24    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
25    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
26    [{'Education 1': {'Education Start 1': '1999.0...  NaN   
27    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
28    [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
29    [{'Education 1': {'Education Start 1': '2008.0...  NaN   
...                                                 ...  ...   
1173  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1174  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1175  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1176  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1177  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1178  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1179  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1180  [{'Education 1': {'Education Start 1': '2008.0...  NaN   
1181  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1182  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1183  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1184  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1185  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1186  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1187  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1188  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1189  [{'Education 1': {'Education Start 1': '2009.0...  NaN   
1190  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1191  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1192  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1193  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1194  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1195  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1196  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1197  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1198  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1199  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1200  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1201  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   
1202  [{'Education 1': {'Education Start 1': 'nan', ...  NaN   

                                 location  \
0     Bangkok Metropolitan Area

In [37]:
for i in range(len(drop_data)):
    loca = drop_data.iloc[i]["location"]
    sloca = loca.split(" ")
    drop_data.iloc[i] = drop_data.iloc[i].set_value("location", sloca[-1])
drop_data
del drop_data["title"]

In [38]:
len(drop_data)

1203

In [39]:
count_skill = {}
new_data = drop_data.copy()
count = 0
for i in range(len(new_data)):
    skills = new_data.iloc[i]["skills"]
    for skill in skills:     
        k = str(list(skill.keys())[0])
        v = int(list(skill.values())[0])
        if k not in count_skill:
            count_skill[k] = 1
            count += 1
        else:
            count_skill[k] += 1
inData = {}
# print(count_skill)
for i in range(len(new_data)):
    skills = new_data.iloc[i]["skills"]
    print(i)
    for skill in skills:     
        k = str(list(skill.keys())[0])
        v = int(list(skill.values())[0])
        if count_skill[k] > 10:
            if k not in inData:
                new_data.insert(0, k, 0, allow_duplicates=False)
                inData[k] = 1
            new_data.iloc[i] = new_data.iloc[i].set_value(k, 1)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [40]:
skilled_data = new_data.copy()
skilled_data.head()
# drop_data.to_csv("full_candidate.csv")

MVC  Contract Management  MS Project  Swift  Valuation  Technical Support  \
0    0                    0           0      0          0                  0   
1    0                    0           0      0          0                  0   
2    0                    0           0      0          0                  0   
3    0                    0           0      0          0                  0   
4    0                    0           0      0          0                  0   

   UML  Customer Relations  Insurance  3G  \
0    0                   0          0   0   
1    0                   0          0   0   
2    0                   0          0   0   
3    0                   0          0   0   
4    0                   0          0   0   

                         ...                          Entrepreneurship  \
0                        ...                                         1   
1                        ...                                         0   
2                        ...                                         0   
3                        ...                                         0   
4                        ...                                         1   

   Management  Business Strategy  Start-ups  Strategy  \
0           1                  1          1         1   
1           1                  0          0         0   
2           1                  0          0         0   
3           1                  0          0         0   
4           1                  1          1         1   

                                          educations  jobs  location  \
0  [{'Education 1': {'Education Start 1': 'nan', ...   NaN  Thailand   
1  [{'Education 1': {'Education Start 1': 'nan', ...   NaN  Thailand   
2  [{'Education 1': {'Education Start 1': 'nan', ...   NaN  Thailand   
3  [{'Education 1': {'Education Start 1': 'nan', ...   NaN  Thailand   
4  [{'Education 1': {'Education Start 1': 'nan', ...   NaN  Thailand   

                                       organizations  \
0  [{'Organization 1': {'Organization End 1': 'Ap...   
1  [{'Organization 1': {'Organization End 1': 'Ma...   
2  [{'Organization 1': {'Organization End 1': 'Oc...   
3  [{'Organization 1': {'Organization End 1': 'Ju...   
4  [{'Organization 1': {'Organization End 1': 'Ja...   

                                              skills  
0  [{'Strategy': 98}, {'Start-ups': 93}, {'Busine...  
1  [{'Management': 14}, {'Social Media': 14}, {'M...  
2  [{'Sales': 4}, {'Logistics': 4}, {'Management'...  
3  [{'Food': 10}, {'Marketing Strategy': 9}, {'Ma...  
4  [{'FinTech': 42}, {'Biometrics': 41}, {'Tokeni...  

[5 rows x 400 columns]

In [41]:
del skilled_data["skills"]
del skilled_data["organizations"]
del skilled_data["educations"]

In [42]:
# 3.5 get the resulting matrix are the one-hot encoded of candidates data
skilled_data

MVC  Contract Management  MS Project  Swift  Valuation  \
0       0                    0           0      0          0   
1       0                    0           0      0          0   
2       0                    0           0      0          0   
3       0                    0           0      0          0   
4       0                    0           0      0          0   
5       0                    0           0      0          0   
6       0                    0           0      0          0   
7       0                    0           0      0          0   
8       0                    0           0      0          0   
9       0                    0           0      0          0   
10      0                    0           0      0          0   
11      0                    0           0      0          0   
12      0                    0           0      0          0   
13      0                    0           0      0          0   
14      0                    0           0      0          0   
15      0                    0           0      0          0   
16      0                    0           0      0          0   
17      0                    0           0      0          0   
18      0                    0           0      0          0   
19      0                    0           0      0          0   
20      0                    0           0      0          0   
21      0                    0           0      0          0   
22      0                    0           0      0          0   
23      0                    0           0      0          0   
24      0                    0           0      0          0   
25      0                    0           0      0          0   
26      0                    0           0      0          0   
27      0                    0           0      0          0   
28      0                    0           0      0          0   
29      0                    0           0      0          0   
...   ...                  ...         ...    ...        ...   
1173    0                    0           0      0          0   
1174    0                    0           0      0          0   
1175    0                    0           0      0          0   
1176    0                    0           0      0          0   
1177    0                    0           0      0          0   
1178    0                    0           0      0          0   
1179    0                    0           0      0          0   
1180    0                    0           0      0          0   
1181    0                    0           0      0          0   
1182    0                    0           0      0          0   
1183    0                    0           0      0          0   
1184    0                    0           0      0          0   
1185    0                    0           0      0          0   
1186    0                    0           0      0          0   
1187    0                    0           0      0          0   
1188    0                    0           0      0          0   
1189    0                    0           0      0          0   
1190    0                    0           0      0          1   
1191    0                    0           0      0          0   
1192    0                    0           0      0          0   
1193    0                    0           0      0          0   
1194    0                    0           0      0          0   
1195    0                    0           0      0          0   
1196    0                    0           0      0          0   
1197    0                    0           0      0          0   
1198    0                    0           0      0          0   
1199    0                    0           0      0          0   
1200    0                    1           0      0          0   
1201    0                    0           0      0          0   
1202    0                    0           0      0          0   

      Technical Support  UML  Custome

In [43]:
loc_data = skilled_data.copy()
for i in range(len(loc_data)):
    locs = new_data.iloc[i]["location"]
    print(i)
    if locs not in loc_data:
        loc_data.insert(0, locs, 0, allow_duplicates=False)
    loc_data.iloc[i] = loc_data.iloc[i].set_value(locs, 1)
loc_data

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Other  California  Hungary  Poland  Singapore  India  Belarus  Kingdom  \
0         0           0        0       0          0      0        0        0   
1         0           0        0       0          0      0        0        0   
2         0           0        0       0          0      0        0        0   
3         0           0        0       0          0      0        0        0   
4         0           0        0       0          0      0        0        0   
5         0           0        0       0          0      0        0        0   
6         0           0        0       0          0      0        0        0   
7         0           0        0       0          0      0        0        0   
8         0           0        0       0          0      0        0        0   
9         0           0        0       0          0      0        0        0   
10        0           0        0       0          0      0        0        0   
11        0           0        0       0          0      0        0        0   
12        0           0        0       0          0      0        0        0   
13        0           0        0       0          0      0        0        0   
14        0           0        0       0          0      0        0        0   
15        0           0        0       0          0      0        0        0   
16        0           0        0       0          0      0        0        0   
17        0           0        0       0          0      0        0        0   
18        0           0        0       0          0      0        0        0   
19        0           0        0       0          0      0        0        0   
20        0           0        0       0          0      0        0        0   
21        0           0        0       0          0      0        0        0   
22        0           0        0       0          0      0        0        0   
23        0           0        0       0          0      0        0        0   
24        0           0        0       0          0      0        0        0   
25        0           0        0       0          0      0        0        0   
26        0           0        0       0          0      0        0        0   
27        0           0        0       0          0      0        0        0   
28        0           0        0       0          0      0        0        0   
29        0           0        0       0          0      0        0        0   
...     ...         ...      ...     ...        ...    ...      ...      ...   
1173      0           0        0       0          0      0        0        0   
1174      0           0        0       0          0      0        0        0   
1175      0           0        0       0          0      0        0        0   
1176      0           0        0       0          0      0        0        0   
1177      0           0        0       0          0      0        0        0   
1178      0           0        0       0          0      0        0        0   
1179      0           0        0       0          0      0        0        0   
1180      0           0        0       0          0      0        0        0   
1181      0           0        0       0          0      0        0        0   
1182      0           0        0       0          0      0        0        0   
1183      0           0        0       0          0      0        0        0   
1184      0           0        0       0          0      0        0        0   
1185      0           0        0       0          0      0        0        0   
1186      0           0        0       0          0      0        0        0   
1187      0           0        0       0          0      0        0        0   
1188      0           0        0       0          0      0        0        0   
1189      0           0        0       0          0      0        0        0   
1190      0           0        0       0          0      0        0        0   
1191  

# 4. Final Data

In [44]:
final_data = loc_data.copy()

In [45]:
del final_data["location"]

# final_data.to_csv("full_candidate_final.csv")